In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType,BooleanType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/16 16:24:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')

In [4]:
df_product.createOrReplaceTempView("product")
df_comment.createOrReplaceTempView("comment")
df_shop.createOrReplaceTempView("shop")

In [5]:
df_product.printSchema()

root
 |-- id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- rating_average: float (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- specifications: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_store_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- category_id: integer (nullable = true)



In [6]:
df_shop.printSchema()

root
 |-- id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = true)
 |-- avg_rating_point: float (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- total_follower: integer (nullable = true)
 |-- days_since_joined: integer (nullable = true)



In [19]:
df_comment.count()

4243515

In [20]:
df_product.count()

46721

In [22]:
spark.sql("""
    select 1 from product p
    left join shop s on p.seller_id = s.id and p.seller_store_id = s.store_id
    left join comment c on c.product_id = p.id
""").count()

4241625

In [25]:
spark.sql("""
    select count(id),count(product_id),count(*) from product
""").show()

+---------+-----------------+--------+
|count(id)|count(product_id)|count(1)|
+---------+-----------------+--------+
|    46721|            46720|   46721|
+---------+-----------------+--------+



In [27]:
df_comment.printSchema()

root
 |-- id: integer (nullable = true)
 |-- contentRaw: string (nullable = true)
 |-- content: string (nullable = true)
 |-- thank_count: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- spid: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_full_name: string (nullable = true)
 |-- purchased_at: string (nullable = true)
 |-- date_purchased_at: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- lable: integer (nullable = true)



In [8]:
spark.sql("""
    select * from comment c
    left join shop s on c.seller_id = s.id
    left join product p on c.product_id = p.id
""").first()

Row(id=13707516, contentRaw='Thấy mọi người khen nó quá nhiều nên lên đây chê 1 tí\n- Thứ nhất là chất âm, do cơ bản nó ko thiên về dải nào nên nghe cứ bằng bằng ko ấn tượng gì, bạn nên cài app chỉnh EQ của nó, nâng tầm trải nghiệm ngay lập tức (EQ preset Bright khá tốt)\n- Nhét vào tai hơi khó, cơ bản phải kín mới đạt được chống ồn tốt, mình nhét cũng phải xoay đi xoay lại vài vòng mới khít, đấy là tai mình to còn vừa, còn tai các bạn nữ nhỏ xinh như bạn gái mình là chịu luôn, quá to. Nghe mọi người bảo nên mua thêm tip spinfit 360, chắc sẽ thử xem\n- Các tính năng phụ như tự động chuyển đổi noise cancellling, speak-to-chat dùng mình thấy phiền nhiều hơn có ích, nên tắt để tiết kiệm pin\nCòn lại hoàn hảo :)))', content='thấy mọi người khen nó quá nhiều nên lên đây chê tí thứ nhất là chất âm do cơ bản nó ko thiên về dải nào nên nghe cứ bằng bằng ko ấn tượng gì bạn nên cài app chỉnh eq của nó nâng tầm trải nghiệm ngay lập tức eq preset bright khá tốt nhét vào tai hơi khó cơ bản phải kín

In [10]:
spark.sql("""
    select * from product
   where id = 20557588
""").collect()

[Row(id=20557588, master_id=20557588, sku='6072702394231', price=5159000, list_price=6490000, original_price=6490000, discount=1331000, discount_rate=21.0, rating_average=4.099999904632568, review_count=39, productset_group_name='Điện Tử - Điện Lạnh/Tủ lạnh', all_time_quantity_sold=169, name='tủ lạnh toshiba inverter lít gr bvu ukg', description=' thiết kế sang trọng nhỏ gọn tủ lạnh inverter toshiba gr bvu ukg là dòng tủ lạnh tủ ngăn đá trên sở hữu thiết kế nhỏ gọn nhưng không kém phần sang trọng với tông màu đen huyền nổi bật toshiba gr bvu ukg có dung tích lít phù hợp với gia đình nhỏ tầm người nhưng sẽ là nội thất bắt mắt trong không gian nhà bạn công nghệ biến tần inverter tiết kiện điện năng tủ lạnh toshiba gr bvu ukg được trang bị công nghệ biến tần inverter kiểm soát sự hoạt động của máy nén giúp duy trì nhiệt độ bên trong tủ và hạn chế tiếng ồn đến mức tối đa công nghệ này sẽ giúp bạn bớt lo lắng về tiền điện mỗi tháng vì mang lại khả năng siêu tiết điện lên đến so với chiếc tủ

In [13]:
spark.sql("""
    select distinct seller_id from product
   where id = 20557588
""").collect()

[Row(seller_id='1')]

In [12]:
spark.sql("""
    select distinct seller_id from comment
   where product_id = 20557588
""").collect()

[Row(seller_id='218'),
 Row(seller_id='75501'),
 Row(seller_id='219316'),
 Row(seller_id='1'),
 Row(seller_id='152033'),
 Row(seller_id='120451'),
 Row(seller_id='11315'),
 Row(seller_id='118795'),
 Row(seller_id='2194'),
 Row(seller_id='7967')]

In [ ]:
20557588